# Model creation and data split-up

This notebook will:
* Split data ito test and train data
* Train the model
* Save the model

Import packages

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pds

Import data

In [2]:
df = pds.read_pickle('ANN_dataset.pkl')
df.head()

,timestamp,lon,lat,direction,speed,journey,seg,time_left,segment_time,tsjs
0,2018-02-16 04:48:40+01:00,58.414238,15.571015,147.300003,0.0,1,1,71.0,71.0,0.0
1,2018-02-16 04:48:41+01:00,58.414246,15.571012,147.300003,0.0,1,1,70.0,71.0,0.0
2,2018-02-16 04:48:42+01:00,58.414249,15.571008,147.300003,0.0,1,1,69.0,71.0,0.0
3,2018-02-16 04:48:43+01:00,58.414257,15.571004,147.300003,0.0,1,1,68.0,71.0,0.0
4,2018-02-16 04:48:44+01:00,58.414257,15.571006,147.300003,0.0,1,1,67.0,71.0,0.0


Note that `time_left` is the label in this model 

In [3]:
label_string = 'time_left'

One hot encode segments, normalize columns and create final dataset

In [4]:
# Make segments categorical
dataset = pds.get_dummies(df['seg'])

# Combine segments, timestamps and time from journey start

# Cyclical timestamps 
dataset['hr_sin'] = np.sin(df.timestamp.dt.hour*(2.*np.pi/24))
dataset['hr_cos'] = np.cos(df.timestamp.dt.hour*(2.*np.pi/24))

# Convert to radians befor trigonometric functions
dataset['dir_sin'] = np.sin(df.direction*(np.pi/180))
dataset['dir_cos'] = np.cos(df.direction*(np.pi/180))

dataset['speed'] = (df['speed']-df['speed'].min())/(df['speed'].max()-df['speed'].min())
dataset['tsjs'] = (df['tsjs']-df['tsjs'].min())/(df['tsjs'].max()-df['tsjs'].min())

dataset['lat'] = (df['lat']-df['lat'].min())/(df['lat'].max()-df['lat'].min())
dataset['lon'] = (df['lon']-df['lon'].min())/(df['lon'].max()-df['lon'].min())

# Include journey number to select entire journeys
dataset['journey'] = df['journey']

dataset.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,dir_sin,dir_cos,speed,tsjs,lat,lon,journey
0,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326317,0.924521,1
1,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326237,0.925287,1
2,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326156,0.925670,1
3,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326076,0.926437,1
4,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326116,0.926437,1


Select ~20% test journeys

In [5]:
divider = 0.2
num_journeys = dataset.journey.unique()[-1]
num_test_segments = np.int_(np.round(num_journeys*divider))
msk = np.random.randint(1, num_journeys, num_test_segments)
mask = dataset['journey'].isin(msk)

Divide training and test data, where data is a sequence from a journey

In [6]:
train_input = dataset[~mask]
test_input = dataset[mask]

train_labels = df[label_string][~mask]
test_labels = df[label_string][mask]

In [7]:
# Journey numbers were needed up until this point
train_data = train_input.drop(columns=['journey'])
test_data = test_input.drop(columns=['journey'])

Build model

In [8]:
model = keras.Sequential([
	keras.layers.Dense(2*len(train_data.columns), activation=tf.nn.sigmoid, input_shape=(train_data.shape[1],)),
    keras.layers.Dense(len(train_data.columns), activation=tf.nn.relu),
	keras.layers.Dense(1, activation=tf.nn.relu)
	])

optimizer = keras.optimizers.Adadelta()
model.compile(loss='mae', optimizer = optimizer, metrics=['mae'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 38)                760       
_________________________________________________________________
dense_1 (Dense)              (None, 19)                741       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 20        
Total params: 1,521
Trainable params: 1,521
Non-trainable params: 0
_________________________________________________________________
None


Fit model, 10% of the training data is used as validation set

In [16]:
EPOCHS=30
result = model.fit(train_data.values, 
                   train_labels.values, 
                   epochs = EPOCHS, 
                   validation_split=0.1, 
                   initial_epoch=15
                   batch_size=128
                  )

Train on 2437120 samples, validate on 270792 samples
Epoch 16/30
2437120/2437120 [==============================] - 64s 26us/step - loss: 11.3160 - mean_absolute_error: 11.3160 - val_loss: 11.3113 - val_mean_absolute_error: 11.3113
Epoch 17/30
2437120/2437120 [==============================] - 64s 26us/step - loss: 11.3007 - mean_absolute_error: 11.3007 - val_loss: 11.4028 - val_mean_absolute_error: 11.4028
Epoch 18/30
2437120/2437120 [==============================] - 62s 26us/step - loss: 11.2850 - mean_absolute_error: 11.2850 - val_loss: 11.3111 - val_mean_absolute_error: 11.3111
Epoch 19/30
2437120/2437120 [==============================] - 63s 26us/step - loss: 11.2664 - mean_absolute_error: 11.2664 - val_loss: 11.2678 - val_mean_absolute_error: 11.2678
Epoch 20/30
2437120/2437120 [==============================] - 65s 27us/step - loss: 11.2496 - mean_absolute_error: 11.2496 - val_loss: 11.3280 - val_mean_absolute_error: 11.3280
Epoch 21/30
2437120/2437120 [=======================

Evaluate model on test data

In [15]:
model.evaluate(test_data.values, test_labels.values)

604088/604088 [==============================] - 7s 11us/step


[11.04705225947588, 11.04705225947588]

Prepare data for the evaluation step

In [11]:
# Same name for all files
model_name = 'ANN_M2'

# serialize model to YAML
model_yaml = model.to_yaml()
with open(model_name + '.yaml', 'w') as yaml_file:
    yaml_file.write(model_yaml)

# Serialize weights to HDF5
model.save_weights(model_name + '.h5')

In [12]:
output = pds.concat([test_input, test_labels], axis=1)
output.head()

,1,2,3,4,5,6,7,8,9,10,...,hr_sin,hr_cos,dir_sin,dir_cos,speed,tsjs,lat,lon,journey,time_left
641,1,0,0,0,0,0,0,0,0,0,...,0.965926,0.258819,0.543174,-0.83962,0.0,0.0,0.326076,0.923372,2,191.0
642,1,0,0,0,0,0,0,0,0,0,...,0.965926,0.258819,0.543174,-0.83962,0.0,0.0,0.326076,0.923372,2,190.0
643,1,0,0,0,0,0,0,0,0,0,...,0.965926,0.258819,0.543174,-0.83962,0.0,0.0,0.326096,0.923372,2,189.0
644,1,0,0,0,0,0,0,0,0,0,...,0.965926,0.258819,0.543174,-0.83962,0.0,0.0,0.326096,0.923755,2,188.0
645,1,0,0,0,0,0,0,0,0,0,...,0.965926,0.258819,0.543174,-0.83962,0.0,0.0,0.326096,0.923755,2,187.0


In [13]:
output.to_pickle(model_name + '.pkl')

Done! Get ready for evaluation